Aggregate fine-tuned model predictions

In [26]:
import pandas as pd
import numpy as np

In [83]:
prefix=r'/prob_finetune_results/'

In [89]:
label=3
file=prefix+"prob_%d.csv" % label

In [90]:
df=pd.read_csv(file)

In [84]:
# average or take the max of prediction of each restaurant for each label
numlabel=9
mean_prob_list=[0 for _ in range(numlabel)]
max_prob_list=[0 for _ in range(numlabel)]
for label in range(numlabel):
    file=prefix+"prob_%d.csv" % label
    df=pd.read_csv(file)
    mean_prob_list[label]=df.groupby(['business_id'])['prob'].mean()
    max_prob_list[label]=df.groupby(['business_id'])['prob'].max()

In [85]:
mean_prob = pd.concat(mean_prob_list, axis=1)
mean_prob.columns=np.arange(numlabel).astype(str)
mean_prob[:3]

,0,1,2,3,4,5,6,7,8
business_id,,,,,,,,,
003sg,0.112358,0.689360,0.428090,0.698655,0.489160,0.666485,0.835466,0.432485,0.628762
00er5,0.100349,0.677469,0.417726,0.686965,0.454925,0.676673,0.845139,0.396486,0.658122
00kad,0.094811,0.661222,0.383829,0.701736,0.426829,0.648342,0.804306,0.391289,0.633757


In [86]:
mean_pred = mean_prob.round()

In [44]:
mean_pred.to_csv(r'/prob_finetune_results/mean_pred.csv',float_format='%d')

In [46]:
max_prob = pd.concat(max_prob_list, axis=1)
max_prob.columns=np.arange(numlabel).astype(str)
max_prob[:3]

,0,1,2,3,4,5,6,7,8
business_id,,,,,,,,,
003sg,0.861286,0.999123,0.996100,0.959985,0.994618,0.999845,0.999967,0.992614,0.996199
00er5,0.535428,0.996083,0.995542,0.960053,0.979293,0.993813,0.997574,0.989250,0.986816
00kad,0.401102,0.997474,0.992476,0.975356,0.986870,0.998778,0.993389,0.963597,0.986858


In [47]:
max_pred = max_prob.round()

In [48]:
max_pred.to_csv(r'/prob_finetune_results/max_pred.csv',float_format='%d')

In [49]:
# mix the max and mean predictions
mix_pred = mean_pred.copy()

In [51]:
mix_pred['3'] = max_pred['3']

In [52]:
mix_pred.to_csv(r'/prob_finetune_results/mix_pred.csv',float_format='%d')

In [87]:
mix_pred[:3]

,0,1,2,3,4,5,6,7,8
business_id,,,,,,,,,
003sg,0,1,0,1,0,1,1,0,1
00er5,0,1,0,1,0,1,1,0,1
00kad,0,1,0,1,0,1,1,0,1


In [54]:
mean_pred[:3]

,0,1,2,3,4,5,6,7,8
business_id,,,,,,,,,
003sg,0,1,0,1,0,1,1,0,1
00er5,0,1,0,1,0,1,1,0,1
00kad,0,1,0,1,0,1,1,0,1


In [82]:
# convert mean prediction to submittable form
with open(r'/prob_finetune_results/mean_submission.csv','w') as fw:
    fw.write('business_id,labels')
    for ind in mean_pred.index:
        pred = list(mean_pred.loc[ind,:].astype(int))
        label = [str(i) for i in range(9) if pred[i]!=0]
        line = ind+','+' '.join(label)+'\n'
        fw.write(line)